## Implementation

### benchmark: $e^{iH\delta t}$ using scipy.linalg.expm

In [1]:
import numpy as np
from src.subcircuit import *

we want to implement the Himiltonian $H = Z_1 \otimes Z_2 \otimes Z_3 \otimes Z_4$

the simulated circuit is therefore represented by $e^{itZ^{\otimes 4}}$

In [2]:
q0 = np.array([1, 0])
q1 = np.array([1, 0]) 
q2 = np.array([0, 1])
q3 = np.array([0, 1])

y0 = np.kron(np.kron(np.kron(q0, q1), q2), q3)

t = 1.086 

H = np.kron(np.kron(np.kron(Z, Z), Z), Z)

theoretical_result = np.abs(np.dot(expm(1j * t * H), y0))**2
print(theoretical_result)

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Implementation using the subcircuit model

target circuit: $U(t) = e^{itZ^{\otimes 4}}$

### Depth 5 Decomposition

$U(t) = e^{it_1^b Y_2Z_3Z_4}e^{i\phi Z_1X_2}e^{it_1^b Y_2Z_3Z_4}e^{i\phi Z_1X_2}e^{it_1^b Y_2Z_3Z_4}$

where $Y_2$ denotes the Pauli-Y gate acting on the 2nd qubit, ...

decomposition of the 3-local Hamiltonian $e^{it_1^b Y_2Z_3Z_4}$ with Hamiltonian H:

$H = Y_2 \otimes Z_3 \otimes Z_4$

$Y_2 = R_{Z,2}(\frac{\pi}{2})HZH R_{Z,2}(\frac{-\pi}{2})$

$H = R_{Z,2}(\frac{\pi}{2})HZ_2H R_{Z,2}(\frac{-\pi}{2}) \otimes Z_3 \otimes Z_4$

$U(t) = e^{it_1^bR_{Z,2}(\frac{\pi}{2})HZ_2H R_{Z,2}(\frac{-\pi}{2}) \otimes Z_3 \otimes Z_4}e^{i\phi Z_1X_2} e^{it_1^bR_{Z,2}(\frac{\pi}{2})HZ_2H R_{Z,2}(\frac{-\pi}{2}) \otimes Z_3 \otimes Z_4}e^{i\phi Z_1X_2}e^{R_{Z,2}(\frac{\pi}{2})HZ_2H R_{Z,2}(\frac{-\pi}{2}) \otimes Z_3 \otimes Z_4}$

with $X = HZH$ all gates have the form: $e^{itZ\otimes Z}$ for arbitrary values of $t$.
Therefore we decomposed all k-local Hamiltonians into single-qubit unitaries.

### Define the initial qubits


$y_0 = q_1 \otimes q_2 \otimes q_3 \otimes q_4$ 

In [3]:
t = 0.00005
sign = 1
c = 1

q1 = np.array([1, 0])
q2 = np.array([1, 0]) 
q3 = np.array([0, 1])
q4 = np.array([0, 1])

y0 = np.kron(np.kron(np.kron(q1, q2), q3), q4)

$t_1^a = t1[0]$ and $t_1^b = t1[1]$

$t_2^a = t2[0]$ and $t_2^b = t2[1]$

### Apply $e^{it_1^bR_{Z,1}(\frac{\pi}{2})HZH R_{Z,1}(\frac{-\pi}{2}) \otimes Z_3 \otimes Z_4}$ to q2-q4

In [4]:
y0_1 = yzz_t1_circuit(t, sign, c, y0)
print(np.abs(y0_1)**2)

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### Apply $e^{i\phi Z_1X_2}$ to q1-q2

In [5]:
y0_2 = zx_circuit(t, y0_1)
print(np.abs(y0_2)**2)

[0.         0.         0.         0.99825663 0.         0.
 0.         0.00174337 0.         0.         0.         0.
 0.         0.         0.         0.        ]


### Apply $e^{it_2^bR_{Z,1}(\frac{\pi}{2})HZH R_{Z,1}(\frac{-\pi}{2}) \otimes Z_3 \otimes Z_4}$ to q2-q4

In [6]:
y0_3 = yzz_t2_circuit(t, sign, c, y0_2)

print(np.abs(y0_3)**2)

[0.         0.         0.         0.99825663 0.         0.
 0.         0.00174337 0.         0.         0.         0.
 0.         0.         0.         0.        ]


### Apply $e^{i\phi Z_1X_2}$ to q1-q2

In [7]:
y0_4 = zx_circuit(t, y0_3)

print(np.abs(y0_4)**2)

[0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.56249961e-10
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]


### Apply $e^{it_1^bR_{Z,1}(\frac{\pi}{2})HZH R_{Z,1}(\frac{-\pi}{2}) \otimes Z_3 \otimes Z_4}$ to q2-q4

In [8]:
y0_5 = yzz_t1_circuit(t, sign, c, y0_4)

practical_result = np.abs(y0_5)**2
print(practical_result)

[0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.56249961e-10
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]


In [9]:
print(theoretical_result - practical_result)

[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.56249458e-10
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.56249961e-10
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
